In [1]:
from os import path
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from lightfm import LightFM
from lightfm.evaluation import auc_score
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import hstack, identity
from scipy.sparse import coo_matrix
from functools import reduce

/anaconda3/lib/python3.6/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [276]:
ratingss = pd.read_csv("ratings.csv")

In [185]:

movies = pd.read_csv("movies.csv")

In [139]:
# добаввляю тэги
tags = pd.read_csv("tags.csv")

In [6]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [186]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [187]:
tags.head()

,userId,movieId,tag,timestamp
0,1,318,narrated,1425942391
1,20,4306,Dreamworks,1459855607
2,20,89302,England,1400778834
3,20,89302,espionage,1400778836
4,20,89302,jazz,1400778841


In [8]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45843 entries, 0 to 45842
Data columns (total 3 columns):
movieId    45843 non-null int64
title      45843 non-null object
genres     45843 non-null object
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [9]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26024289 entries, 0 to 26024288
Data columns (total 4 columns):
userId       int64
movieId      int64
rating       float64
timestamp    int64
dtypes: float64(1), int64(3)
memory usage: 794.2 MB


In [411]:
ratings =ratingss.copy()

In [412]:
#ratings=ratings.loc[:2438890]

In [413]:
ratings["movie_id"] = ratings["movieId"].astype("category").cat.codes.copy()
ratings["user_id"] = ratings["userId"].astype("category").cat.codes.copy()
last_movie_id = ratings["movie_id"].max()
last_user_id = ratings["user_id"].max()

In [420]:
# both matrix with rank >=4

In [415]:
#ratings=ratings.loc[:25308]

In [416]:
user_with_item = ratings[["user_id", "movie_id"]].as_matrix()
user_item_matrix = csr_matrix(((ratings["rating"] >=4.0).tolist(), ([both[0] for both in user_with_item],
                                                                [both[1] for both in user_with_item],)),
    shape=(last_user_id + 1, last_movie_id + 1),dtype=np.float32)

In [417]:
np.random.seed(0)
test_indices = np.random.choice(
    range(user_item_matrix.nnz), replace=False, size=int(user_item_matrix.nnz * 0.7)).tolist()

In [418]:
train = user_item_matrix.copy()
train.data[test_indices] = 0
train.eliminate_zeros()

test = user_item_matrix.copy()
test.data[:] = 0
test.data[test_indices] = user_item_matrix.data[test_indices]
test.eliminate_zeros()

In [16]:
# preparing genres for features

In [213]:
movies_genres = ratings[["movieId", "movie_id"]].drop_duplicates().join(
    movies, on="movieId", rsuffix="codes", lsuffix="movies", sort=True).fillna("None")[["movie_id", "genres"]]
movies_genres["genres_set"] = movies_genres["genres"].apply(lambda x: set(x.split("|")))

In [214]:
movies_genres.loc[movies_genres["genres"] == "(no genres listed)", "genres"] = "None"
movies_genres["genres"] = movies_genres["genres"].apply(lambda x: x.replace("-", ""))

movies_features = CountVectorizer().fit_transform(movies_genres["genres"])
movies_features.nnz

55855

In [153]:
#tagss=tags[["movieId"]].drop_duplicates()

In [ ]:
# preparing tags for features

In [398]:
movies_tags = tags.join(
    movies, on="movieId", rsuffix="codes", lsuffix="tags", sort=True).fillna("None")[["movieId","tag"]]

In [399]:
movies_tags["genre_tags"] = movies_tags["tag"].apply(lambda x: set(x.split("|")))

In [197]:
movies_tags=movies_tags.drop_duplicates(subset=['movieId'])

In [400]:
movies_tags=movies_tags.fillna("None")[["movieId","tag",'genre_tags']]

In [401]:
movies_tags.head()

,movieId,tag,genre_tags
3420,1,Pixar,{Pixar}
3421,1,time travel,{time travel}
4336,1,computer animation,{computer animation}
4337,1,funny,{funny}
4338,1,Pixar,{Pixar}


In [402]:
movies_tags.loc[movies_tags["genre_tags"] == "(no genre_tags listed)", "genre_tags"] = "None"
movies_tags["tag"] = movies_tags["tag"].apply(lambda x: x.replace("-", ""))

movies_features_tags = CountVectorizer().fit_transform(movies_tags["tag"])
movies_features_tags.nnz

1212279

In [403]:
features_genres = hstack([identity(movies_genres.shape[0]), movies_features])
features_tag = hstack([identity(movies_tags.shape[0]), movies_features_tags])

In [404]:
features_tag.nnz

1965449

In [405]:
features_genres.nnz

100970

In [232]:
test.nnz

129881

In [233]:
train.nnz

12851861

In [209]:
from scipy.sparse import save_npz

save_npz("train.npz", train)
save_npz("test.npz", test)
save_npz("features_tag.npz", features_tag)
save_npz("features_genres.npz", features_genres)

In [62]:
# fit model without features

In [426]:
%%time
model_without_features = LightFM(loss="bpr")
model_without_features.fit(train, num_threads=4)

train_mod = auc_score(model_without_features, train).mean()
test_mod = auc_score(model_without_features, test).mean()
print('ROC AUC: train %.2f, test %.2f.' % (train_mod, test_mod))

ROC AUC: train 0.69, test 0.68.
CPU times: user 25min 57s, sys: 12.4 s, total: 26min 9s
Wall time: 26min 53s


In [61]:
# fit model with features ( genres )

In [424]:
%%time 
model_with_features = LightFM(loss="bpr")
model_with_features.fit(train, num_threads=4, item_features=features)

train_mod_feature = auc_score(model_with_features, train, item_features=features).mean()
test_mod_feature = auc_score(model_with_features, test, item_features=features).mean()
print('ROC AUC: train %.2f, test %.2f.' % (train_mod_feature, test_mod_feature))

ROC AUC: train 0.82, test 0.80.
CPU times: user 24min 38s, sys: 2.82 s, total: 24min 41s
Wall time: 24min 47s


In [406]:
train.shape, features_tag.shape

((25308, 25118), (753170, 783671))

In [295]:
test.shape

(311, 4694)

In [244]:
features_tag.shape

(25308, 33528)

In [245]:
features_genres.shape

(45115, 45135)

In [425]:
%%time 
model_with_features_tags = LightFM(loss="bpr")
model_with_features_tags.fit(train, num_threads=4, item_features=features_tag)

train_mod_feature_tags = auc_score(model_with_features_tags, train, item_features=features_tag).mean()
test_mod_feature_tags = auc_score(model_with_features_tags, test, item_features=features_tag).mean()
print('ROC AUC: train %.2f, test %.2f.' % (train_mod_feature_tags, test_mod_feature_tags))

ROC AUC: train 0.79, test 0.75.
CPU times: user 32min 32s, sys: 15.1 s, total: 32min 47s
Wall time: 33min 42s


In [428]:
# Better results and time are in model with genres features .  
print('ROC AUC without features: train %.2f, test %.2f.' % (train_mod, test_mod))
print('ROC AUC genres: train %.2f, test %.2f.' % (train_mod_feature, test_mod_feature))
print('ROC AUC tags: train %.2f, test %.2f.' % (train_mod_feature_tags, test_mod_feature_tags))

ROC AUC without features: train 0.69, test 0.68.
ROC AUC genres: train 0.82, test 0.80.
ROC AUC tags: train 0.79, test 0.75.
